In [21]:
import pandas as pd

### Listen Events Table Old

In [ ]:
df = pd.read_csv("/home/lupusruber/music_analytics/data/events.csv", encoding="utf-8")
df.drop(["Unnamed: 0"], inplace=True, axis=1)

In [54]:
transform_string = (
    lambda x: "".join([("_" + i if i.isupper() else i) for i in x]).strip().lower()
)

In [55]:
df.columns = list(map(transform_string, df.columns.tolist()))
df = df.rename({"ts": "timestamp"}, axis=1)

In [56]:
df.head(1)

,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000


In [57]:
df.groupby("session_id").count().reset_index()

,session_id,artist,song,duration,timestamp,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,10,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
3,28,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,31,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14657,55788,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
14658,55805,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
14659,55848,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
14660,55882,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [58]:
df.groupby("gender")["gender"].count() / len(df)

gender
F    0.47126
M    0.52874
Name: gender, dtype: float64

In [59]:
df.query("user_id==748 and item_in_session==30").sort_values(
    "timestamp", ascending=True
).head(10)

,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
650,Irma Thomas,The Same Love That Made Me Laugh [LP Version],195.36934,1616087608000,747,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000
80143,Chris Vickery,Jokers and Thieves,191.71220,1616302962000,49237,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000


In [60]:
result = (
    df.groupby("session_id")
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# result['count'].sum() == n_rows
result

,session_id,count
7220,37432,219
7958,38794,216
13931,51602,199
4659,30891,197
8117,39079,178
...,...,...
4605,30649,1
4637,30787,1
4635,30784,1
4634,30781,1


In [61]:
df.query("session_id == 37432")[
    ["artist", "song", "duration", "item_in_session", "user_id"]
]["user_id"].unique()

array([17692])

In [62]:
session_and_user = (
    df.groupby(["session_id", "user_id"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# df.groupby(['sessionId', 'userId']).size().reset_index(name='count')['count].sum() == n_rows
session_and_user

,session_id,user_id,count
7220,37432,17692,219
7958,38794,17940,216
13931,51602,19676,199
4659,30891,16304,197
8117,39079,17780,178
...,...,...,...
4605,30649,1903,1
4637,30787,10466,1
4635,30784,13727,1
4634,30781,17442,1


In [63]:
session_and_user.groupby("user_id").size().reset_index(name="count").sort_values(
    "count", ascending=False
)

,user_id,count
5726,13473,17
3371,8010,16
121,288,15
3119,7357,13
3783,8943,12
...,...,...
8439,19986,1
8425,19958,1
8426,19959,1
8428,19964,1


In [64]:
session_and_user.sort_values("user_id")

,session_id,user_id,count
4132,28898,3,3
10935,44377,8,1
6093,35076,8,3
0,9,10,1
10626,43735,11,5
...,...,...,...
14085,52141,19996,1
13523,50407,19997,5
2870,19996,19997,1
7136,37261,20001,37


In [65]:
df["user_id"].unique().shape[0] == df["session_id"].unique().shape[0]

False

In [66]:
df.query("user_id == 19997").groupby("session_id").aggregate(lambda x: x)[
    "item_in_session"
]

session_id
19996                  0
50407    [0, 1, 3, 5, 6]
Name: item_in_session, dtype: object

### Auth Events Table

In [35]:
auth_events = pd.read_csv("/home/lupusruber/music_analytics/data/auth_events.csv")
auth_events.drop(["Unnamed: 0"], inplace=True, axis=1)
print(auth_events.head(1)["userAgent"].values[0])

Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:30.0) Gecko/20100101 Firefox/30.0


In [68]:
auth_events.query("success != True").head(3)

,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
44,1616090567000,38169,paid,1,San Antonio,78236,TX,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",-98.618734,29.391283,NaN,NaN,NaN,NaN,NaN,False
45,1616090568000,38169,paid,2,San Antonio,78236,TX,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",-98.618734,29.391283,NaN,NaN,NaN,NaN,NaN,False
62,1616092648000,38122,paid,12,Rancho Cordova,95670,CA,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-121.280327,38.604268,NaN,NaN,NaN,NaN,NaN,False


### Page View Events Table

In [33]:
page_view = pd.read_csv("/home/lupusruber/music_analytics/data/page_view_events.csv")
page_view.drop(["Unnamed: 0", "userAgent"], inplace=True, axis=1)
page_view["ts"] = pd.to_datetime(page_view["ts"], unit="ms")
page_view.sample(10)

,ts,sessionId,page,auth,method,status,level,itemInSession,city,zip,...,lon,lat,userId,lastName,firstName,gender,registration,artist,song,duration
70939,2021-03-20 04:59:35,31705,NextSong,Logged In,PUT,200,paid,46,De Soto,63020,...,-90.565549,38.100971,9333.0,Vazquez,Chloe,F,1.616087e+12,Bob Hope,Merry-Go-Runaround,147.72200
58172,2021-03-19 19:18:44,17430,NextSong,Logged In,PUT,200,paid,22,Issaquah,98027,...,-121.998724,47.493732,17431.0,Potter,Olivia,F,1.616087e+12,Danny Williams,The Wonderful World Of The Young,168.01913
16953,2021-03-19 00:48:17,38831,NextSong,Logged In,PUT,200,paid,65,Chicago,60642,...,-87.658544,41.902042,1675.0,Ford,Luke,M,1.616087e+12,Luis Alberto Spinetta,Alfil_ Ella No Cambia Nada,251.03629
7172,2021-03-18 19:38:16,38870,NextSong,Logged In,PUT,200,paid,1,Massillon,44647,...,-81.565063,40.797864,12539.0,Cooper,Porter,F,1.616087e+12,The Velvet Underground,I Heard Her Call My Name,276.68854
82786,2021-03-20 16:00:48,49388,NextSong,Logged In,PUT,200,paid,25,Tucson,85715,...,-110.836125,32.246750,15591.0,Rubio,Aaron,M,1.616087e+12,Natassa Theodoridou,Den S'Adiko,212.06158
85403,2021-03-20 17:28:55,18847,NextSong,Logged In,PUT,200,paid,27,Battle Creek,49015,...,-85.228740,42.270781,18848.0,King,Mileidy,F,1.616087e+12,Juelz Santana,Lil' Boy Fresh,233.92609
71632,2021-03-20 05:57:18,47881,NextSong,Logged In,PUT,200,free,1,Middletown,7748,...,-74.115274,40.396300,18791.0,Chan,Collier,M,1.616087e+12,Randy Travis,I Am Going (Album Version),228.75383
73413,2021-03-20 08:32:07,28604,Downgrade,Logged In,GET,200,paid,2,Palm Coast,32137,...,-81.219358,29.581655,17764.0,Anderson,Lilly,F,1.616087e+12,NaN,NaN,NaN
70542,2021-03-20 04:26:44,44603,NextSong,Logged In,PUT,200,paid,16,Charlotte,28206,...,-80.820999,35.256127,4699.0,Davis,Faye,F,1.616087e+12,Montserrat CaballÃ©;Placido Domingo;Vicente Sa...,Sono andati? Fingevo di dormire,511.16364
20530,2021-03-19 03:02:52,40337,NextSong,Logged In,PUT,200,paid,10,Pinehurst,28374,...,-79.462959,35.197454,4795.0,Pineda,Aliyah,F,1.616087e+12,James Newton Howard,U North,108.35546


In [70]:
page_view["page"].unique().tolist()

['NextSong',
 'Upgrade',
 'Submit Upgrade',
 'Settings',
 'Home',
 'Login',
 'Logout',
 'Downgrade',
 'About',
 'Help',
 'Error',
 'Save Settings',
 'Submit Downgrade',
 'Cancel',
 'Cancellation Confirmation']

In [71]:
page_view["method"].unique().tolist()

['PUT', 'GET']

In [72]:
page_view.columns.tolist()

['ts',
 'sessionId',
 'page',
 'auth',
 'method',
 'status',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration',
 'artist',
 'song',
 'duration']

### Status Change Events Table

In [14]:
status_change = pd.read_csv(
    "/home/lupusruber/music_analytics/data/status_change_events.csv"
)

status_change.rename({"ts": "timestamp"}, axis=1, inplace=True)
status_change.drop(["Unnamed: 0"], inplace=True, axis=1)

status_change["timestamp"] = pd.to_datetime(status_change["timestamp"], unit="ms")
status_change["registration"] = pd.to_datetime(status_change["registration"], unit="ms")

status_change.sample(3)

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
4108,2021-07-12 11:44:07,726522,Logged In,free,2,Redmond,98052,WA,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-122.120531,47.680990,8201,Davis,Valeria,F,2021-03-18 17:00:00
2412,2021-05-20 16:04:16,420365,Logged In,free,3,La Fayette,30728,GA,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8...",-85.225790,34.684875,1068,Marquez,Thomas,M,2021-03-18 17:00:00
3811,2021-07-01 20:44:21,671379,Logged In,paid,34,Ayer,1432,MA,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,-71.575135,42.566573,10665,Sanchez,Emily,F,2021-03-18 17:00:00


In [74]:
status_change["auth"].unique().tolist()

['Logged In']

In [75]:
status_change.columns.tolist()

['ts',
 'sessionId',
 'auth',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'userAgent',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration']

### Listen Events Table

In [22]:
import pandas as pd

listen_events = pd.read_csv(
    r"/home/lupusruber/music_analytics/data/listen_events.csv", encoding="utf-8"
)
listen_events.rename({"ts": "timestamp"}, axis=1, inplace=True)
listen_events["timestamp"] = pd.to_datetime(listen_events["timestamp"], unit="ms")
listen_events["registration"] = pd.to_datetime(listen_events["registration"], unit="ms")
listen_events.drop(["Unnamed: 0"], inplace=True, axis=1)

In [37]:
listen_events.columns.tolist()

['artist',
 'song',
 'duration',
 'timestamp',
 'sessionId',
 'auth',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration']

In [23]:
listen_events.sample(5)

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,lon,lat,userId,lastName,firstName,gender,registration
39579,Bert Kaempfert And His Orchestra,Wheeling Free,201.92609,2021-03-19 15:55:48,8027,Logged In,paid,42,Columbus,31907,GA,-84.901475,32.482037,8028,Soto,Addison,F,2021-03-18 17:00:00
547,DJ X-Change featuring DJ Joey A,Come Over,184.31955,2021-03-18 17:11:22,13852,Logged In,paid,30,Greenville,45331,OH,-84.650546,40.098606,13853,Mack,Nathaniel,M,2021-03-18 17:00:00
25781,Blind Boy Fuller,Got To Find My Little Woman,151.19627,2021-03-19 09:27:51,2978,Logged In,paid,46,Glenview,60025,IL,-87.821026,42.075201,2979,Williams,Jacob,M,2021-03-18 17:00:00
98008,Kaija Koo,Viimeinen lento,182.33424,2021-03-22 00:52:47,42805,Logged In,paid,4,Boonton,7005,NJ,-74.417304,40.932771,17971,Barrett,Nataly,F,2021-03-18 17:00:00
72208,Champion Jack Dupree,Christina Christina Blues,200.80281,2021-03-20 18:13:35,40044,Logged In,paid,12,Wellesley Hills,2481,MA,-71.275531,42.311927,9886,Walker,Katherine,F,2021-03-18 17:00:00


In [32]:
assert listen_events.query('timestamp <= registration').shape[0] == 0, "Invalid Rows"

In [24]:
listen_events["auth"].unique().tolist()

['Logged In']

In [20]:
!ls -l

total 68
-rw-r--r-- 1 lupusruber lupusruber   814 Sep  2 11:53 consumer.py
-rw-r--r-- 1 lupusruber lupusruber 63467 Sep  3 11:13 tests.ipynb
